# Function Calling
- Function Calling은 OpenAI API에서 특정 함수 정의(JSON schema 기반)를 주고, 모델이 해당 함수를 호출하는 형식의 응답을 생성하게 하는 기능입니다.
- 챗봇, 에이전트, 도구 사용 등 다양한 자동화 및 플러그인 연동에 활용됩니다.

- 지원 모델 gpt-3-turbo, gpt-4, gpt-4-turbo  (2025.1 현재)

- 모델이 우리의 코드를 호출하도록 해서 우리의 함수들을 모델이 호출할수 있도록 하거나
- 모델이 우리가 원하는 특정 모양과 형식의 output 을 갖도록 강제할수 있다.

- https://platform.openai.com/docs/guides/function-calling?api-mode=chat





In [1]:
from langchain_openai.chat_models.base import ChatOpenAI
from langchain_core.prompts.prompt import PromptTemplate


In [2]:
llm = ChatOpenAI(
    temperature=0.1,
)

prompt = PromptTemplate.from_template("Who is the weather in {city}")
chain = prompt | llm

response = chain.invoke({
    'city': 'Rome',
})

print(response.content)

response

I'm sorry, I am an AI assistant and I do not have real-time information. You can check the weather in Rome by using a weather website or app.


AIMessage(content="I'm sorry, I am an AI assistant and I do not have real-time information. You can check the weather in Rome by using a weather website or app.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 13, 'total_tokens': 46, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BoTMm74DzFdc5Z38aTnyXZQjkQXWQ', 'finish_reason': 'stop', 'logprobs': None}, id='run--96f0ad78-9f00-4803-a8ce-20eb04dc6cdd-0', usage_metadata={'input_tokens': 13, 'output_tokens': 33, 'total_tokens': 46, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [ ]:
# 실시간으로 날씨를 가져올수 없단다.
# 하지만!  이제 함수가 있다고 생각해보자.  실시간 데이터를 가져올 수 잇는 함수 말이다!
#  함수에 필요한 정보는 아마 '장소' 정보 뿐 일거다.  (위도, 경도..)

# ↓여기에 함수가 있다고 가정하자

In [12]:
def get_weather(lon, lat):
    print(f'call an api... lon:{lon}, lat:{lat}')

In [ ]:
# gpt-3, gpt-4 에게 우리에게 이러한 일을 해주는 get_weather() 라는 함수가 있다고 말해주는 거다.
# 그러면, gpt 는 우리의 질문을 보게 되겠고, 함수를 사용하는 것이 이런 실망스러운 답변을 주는 것 보다 나은지 판별할거다.

# 어떻게 GPT 나  LLM 을 그렇게 동작하도록 만들수 있을까?
# LLM 이 get_weather 라는 함수가 있다는 걸 알도록 할수 있을까?

# 이 함수를 위한 JSON schema 를 만드는 거다! ↓


# 함수의 스키마

In [3]:
#  함수의 작동방식과 필요한 것을 설명해 주는 스키마.
#  ↓ 
function_schema = {
    "name": "get_weather",  # 함수의 이름
    # 무슨일을 하는 함수인지 기술
    "description": "function that takes longitude and latitude to find the weather of a place",  

    # 파라미터 기술
    "parameters": {
        "type": "object",
        "properties": {
            # 경도 
            "lon": {"type": "string","description": "The longitude coordinate",},
            # 위도
            "lat": {"type": "string","description": "The latitude coordinate",},
        }
    },

    # 필수 사항 기술
    "required": ["lon", 'lat'], 
    
}

# ChatOpenAI 에 함수 전달하기 (알려주기)

In [4]:
llm = ChatOpenAI(
    temperature=0.1
).bind(  # <- ChatOpenAI  에게 전달할 인수를 추가해 줄수 있다.
    functions=[   # <- 원하는 만큼 많은 함수를 전달할수도 있다.
        function_schema  # <- 준비한 함수 스키마 전달
    ],

    # ↓기본적으로 모델이 특정 함수를 사용하도록 '강제'하거나, 
    #   모델이 함수를 사용하도록 하거나 그냥 답변을 할 수 있도록 모델 스스로 선택하게 할수 있다.

    # 모델을 강제로 함수를 사용하도록 하려면! 아래와 같이 해야 한다
    # function_call={  
    #     "name": "get_weather",  # <-- get_weather 함수를 반드시 사용하라!        
    # },

    
    # auto => AI가 필요에 따라 선택하여 사용하도록 함.
    function_call="auto",
)

In [5]:
chain = prompt | llm

response = chain.invoke({
    'city': 'Rome',
})

print(response.content)

response

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"lon":"12.4964","lat":"41.9028"}', 'name': 'get_weather'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 73, 'total_tokens': 97, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BoTZ6CwUeuicgOe9FyLV8jZqvjj7l', 'finish_reason': 'function_call', 'logprobs': None}, id='run--425449e7-22c6-4c9f-a640-541c599931f8-0', usage_metadata={'input_tokens': 73, 'output_tokens': 24, 'total_tokens': 97, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [6]:
response.additional_kwargs

{'function_call': {'arguments': '{"lon":"12.4964","lat":"41.9028"}',
  'name': 'get_weather'},
 'refusal': None}

In [7]:
response.additional_kwargs['function_call']

{'arguments': '{"lon":"12.4964","lat":"41.9028"}', 'name': 'get_weather'}

In [8]:
response.additional_kwargs['function_call']['arguments']

'{"lon":"12.4964","lat":"41.9028"}'

In [9]:
import json
r = json.loads(response.additional_kwargs['function_call']['arguments'])
r

{'lon': '12.4964', 'lat': '41.9028'}

In [10]:
r['lon'], r['lat']

('12.4964', '41.9028')

In [13]:
get_weather(r['lon'], r['lat'])

call an api... lon:12.4964, lat:41.9028


In [14]:
get_weather(**r)

call an api... lon:12.4964, lat:41.9028


# 퀴즈 생성하기 스키마

In [15]:
function_schema = {
    # '퀴즈 생성하기' 스키마 
    "name": "create_quiz",
    # ↓ 완전히 지어낸 함수다!
    "description": "function that takes a list of questions and answers and returns a quiz",  

    "parameters": {
        "type": "object",
        "properties": {
# ↓ 우라기 원하는 답변의 스키마는
 
# 바로 이 형태를 정의하는 거다. 
# {
#   "questions":[
#      0:{
#        "question":"What … in the story?"
#        "answers":[
#           0:{
#             "answer":"John"
#             "correct":false
#           }
#           ... 
#        ]
#      }
#      ...            
# }       

            "questions": {  # questions 는array!
                "type": "array",   
                "items": {
                    "type": "object",
                    "properties": {
                        "question": { # question 은 string
                            "type": "string",
                        },
                        "answers": {  # answers 는 array
                            "type": "array",
                            "items": {  
                                "type": "object",
                                "properties": {
                                    "answer": {   # answer 는 string
                                        "type": "string",
                                    },
                                    "correct": {  # correct 는 boolean
                                        "type": "boolean",
                                    },
                                },
                                "required": ["answer", "correct"],  # answer 와 correct 둘다 필수!
                            },
                        },
                    },
                    "required": ["question", "answers"],  # question 와 anstions 둘다 필수
                },  # end items {}
            }  # end questions {}
            
        } # end properties{}
    }, # end parameters {}

    "required": ["questions"],
}

In [16]:
llm = ChatOpenAI(
    temperature=0.1,
).bind(
    functions=[
        function_schema  # 여기에는 원하는 만큼의 function 들을 적어 넣을수 있다.
    ],
    # function_call="auto"  # "auto" 모델이 우리 함수를 사용할수도 있고 사용하지 않을수도 있다.
    function_call={
        "name": "create_quiz",
    },
)

# prompt 변경
prompt = PromptTemplate.from_template("Make a quiz about {city}")

chain = prompt | llm

response = chain.invoke({
    "city": "rome"
})

response = response.additional_kwargs['function_call']['arguments']

response

'{"questions":[{"question":"What year was Rome founded?","answers":[{"answer":"753 BC","correct":true},{"answer":"476 AD","correct":false},{"answer":"1000 AD","correct":false}]},{"question":"Who was the first emperor of Rome?","answers":[{"answer":"Julius Caesar","correct":false},{"answer":"Augustus","correct":true},{"answer":"Nero","correct":false}]},{"question":"What famous structure in Rome was built by the ancient Romans?","answers":[{"answer":"Eiffel Tower","correct":false},{"answer":"Colosseum","correct":true},{"answer":"Big Ben","correct":false}]}]}'

In [17]:
import pprint
pprint.pprint(json.loads(response))

{'questions': [{'answers': [{'answer': '753 BC', 'correct': True},
                            {'answer': '476 AD', 'correct': False},
                            {'answer': '1000 AD', 'correct': False}],
                'question': 'What year was Rome founded?'},
               {'answers': [{'answer': 'Julius Caesar', 'correct': False},
                            {'answer': 'Augustus', 'correct': True},
                            {'answer': 'Nero', 'correct': False}],
                'question': 'Who was the first emperor of Rome?'},
               {'answers': [{'answer': 'Eiffel Tower', 'correct': False},
                            {'answer': 'Colosseum', 'correct': True},
                            {'answer': 'Big Ben', 'correct': False}],
                'question': 'What famous structure in Rome was built by the '
                            'ancient Romans?'}]}
